In [4]:
text_file = open("trainset.txt", "r")
lines = text_file.readlines()
#print(lines[1])
text_file.close()
#read line by line

In [5]:
listOfAttributes = [] # implicit instantiation
listOfAttributes.append([])
i = 0
while i < len(lines):
    attributes = lines[i].split('\t') #split it by tab
    listOfAttributes.append([])
    listOfAttributes[i].append(attributes[0])
    listOfAttributes[i].append(attributes[1])
    listOfAttributes[i].append(attributes[2])
    listOfAttributes[i].append(attributes[3])
    i+=1
        
#print(listOfAttributes[0])
# print(len(listOfAttributes) )

### Prepossesing

In [9]:
#Doing prepossesing
import re #regex
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def preprocess(eliment):
      
    eliment = re.sub(r'([^\s\w]|_)+', '', eliment) # Remove all the special characters except space
    eliment = eliment.lower()
    eliment = re.sub(r'\s+', ' ', eliment, flags=re.I)# Substituting multiple spaces with single space

    all_words = eliment.split()
    en_stops = set(stopwords.words('english')) #stopword handle
    eliment = ""
    ps = PorterStemmer() #stemming handle  

    for word in all_words: 
        if word not in en_stops:   
            eliment +=  ps.stem(word) +" "

    return eliment 

listOfAttributesProcessed = [] # implicit instantiation
listOfAttributesProcessed.append([])
i = 0
while i <len(lines):
    listOfAttributesProcessed.append([])
    listOfAttributesProcessed[i].append(listOfAttributes[i][0])
    
    processedEliment1= preprocess(listOfAttributes[i][1] )
    processedEliment2= preprocess(listOfAttributes[i][2] )
    processedEliment3= preprocess(listOfAttributes[i][3] )
    
    AllEliments = processedEliment1+processedEliment2+processedEliment3 #combine attributes
    
    listOfAttributesProcessed[i].append(AllEliments)

#    print(listOfAttributesProcessed[i])  
    i+=1

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  Searched in:
    - 'C:\\Users\\gayan/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\share\\nltk_data'
    - 'C:\\ProgramData\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\gayan\\AppData\\Roaming\\nltk_data'
**********************************************************************


In [7]:
import csv

with open("featuresTrain.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(listOfAttributesProcessed)

In [8]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

df=pd.read_csv('featuresTrain.csv',sep=',',names=['class','text'])
df.head();

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Traning and testing data extraction

In [6]:
df_x=df["text"]
df_y=df["class"]
cv = TfidfVectorizer(min_df=1)
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.333, random_state=0)


In [7]:
cv = TfidfVectorizer(max_features=1500, min_df=1, max_df=0.7) #minimum number of documents that should contain this feature
x_traincv=cv.fit_transform(x_train)
a=x_traincv.toarray()

### Predictions

In [10]:
mnb = MultinomialNB()
mnb.fit(x_traincv,y_train);
x_testcv=cv.transform(x_test);
x_testcv.toarray();
predictions=mnb.predict(x_testcv)
#print(predictions);

NameError: name 'x_traincv' is not defined

### Accuracies

In [9]:
actual=np.array(y_test)
print('Training Accuracy: ',mnb.score(x_traincv,y_train))

Training Accuracy:  1.0


In [10]:
# calculate accuracy of class predictions
from sklearn import metrics
acc = metrics.accuracy_score(actual, predictions)
print('Testing Accuracy: ',acc)

Testing Accuracy:  0.9552238805970149


### Confution matrix

In [11]:
print(metrics.confusion_matrix(actual, predictions)) #confusion matrix

[[23  0]
 [ 3 41]]


Confusion matrix 
[TN FP
 FN TP]

### Classification report

In [12]:
target_names =["+1", "-1"]
print(classification_report(actual, predictions, target_names=target_names))

             precision    recall  f1-score   support

         +1       0.88      1.00      0.94        23
         -1       1.00      0.93      0.96        44

avg / total       0.96      0.96      0.96        67



### Testsetwithoutlabels.txt prediction

In [13]:

text_file = open("testsetwithoutlabels.txt", "r")
l = text_file.readlines()
test=cv.transform(l)
test.toarray()
predict=mnb.predict(test)
#print('%d %s' % predict[0],l[0])
i=0
while i < len(l):
    print ("{0} {1}".format(predict[i], l[i]) )
    i+=1



1  USSR WHEAT BONUS OFFER SAID STILL UNDER DEBATE	 WASHINGTON, March 6 -	 The Reagan administration continues  to debate whether to offer subsidized wheat to the Soviet  Union, but would need assurances from the Soviets that they  would buy the wheat before the subsidy offer would be made, a  senior U.S. Agriculture Department official said.  "I think it still is under active debate whether or not it  would be advisable" to make an the export enhancement offer to  the Soviets, Thomas Kay, administrator of the department's  Foreign Agriculture Service, told Reuters.  "We'd need some assurances from them (the Soviets) that they  would buy if offered" the wheat under the subsidy plan, he said.  Kay called reports that such an offer was imminent "premature."  The Reagan administration's cabinet-level Economic Policy  Council is set to meet today to discuss, among other matters,  agricultural policy but is not expected to address a wheat  subsidy offer to the Soviet Union, administration of